# | WEB SCRAPING: Town House

## 1 - Install & Import Libraries

In [ ]:
# Libraries Installation

#!pip install lxml
#!pip install requests
#!pip install beautifulsoup4

In [ ]:
# Import Libraries

import requests
from urllib.request import urlopen

from bs4 import BeautifulSoup
import csv
from itertools import zip_longest


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


#Optional Libraries

import datetime
import time
import smtplib


In [155]:
#-------------------------------------------------- Create Data Lists---------------------------------------------------

#--------- Elements Lists---------

prod_name_elements = []
org_price_elements = []
sale_elements = []
sale_price_elements = []
new_badge_elements = []
availability_elements = []
size_elements = []

#---------- Text Lists-------------

prod_name = []
org_price = []
sale_price = []
new_badge = []
availability = []
size = []
color = []

#----------- Link list------------

link = []




# ------------------------------------------------------FETCH THE DATA----------------------------------------------------



headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
page = requests.get("https://townteam.com/collections/shoes-men", headers=headers)

# Assign Page Source
src = page.content
#print(src)






# -----------------------------------------------------PARSE THE DATA-------------------------------------------

# Create Soup Object for Parsing

ugly_soup = BeautifulSoup(src, "lxml")
soup = BeautifulSoup(ugly_soup.prettify(), "lxml")
soup_skills = BeautifulSoup(ugly_soup.prettify(), "lxml")
#print(soup)







#------------------------Getting The Underlined Elements----------------------

# Check The Container (Product)
products = soup.find_all("li", {"class":"product"})
#print(products)


# Looping in Product Elements ===>> to get the other elements



for li in products:
           
      
    prod_name_elements.append(li.find("a", {"class":"card-title link-underline card-title-ellipsis card-title-change"}))
    #print(prod_name_elements)

    org_price_elements.append(li.find("s",{"class":"price-item price-item--regular"}))
    #print(org_price_elements)

    sale_price_elements.append(li.find("span",{"class":"price-item price-item--sale"}))
    #print(sale_price_elements)

    new_badge_elements.append(li.find("span",{"class":"badge new-badge"}))
    #print(new_badge_elements)

    availability_elements.append(li.find("form",{"action":"/cart/add","method":"post","class":"variants"}))
    #print(availability_elements)




#------------------------Getting The Underlined Information and Filling The Text Lists----------------------
for i in range(len(products)):
    
    
    if prod_name_elements[i] is not None:
        prod_name.append(prod_name_elements[i].text.split()[0])
    else:
        prod_name.append("N/A")
        
        
        
    
    if org_price_elements[i] is not None:
        org_price.append(org_price_elements[i].text.strip())
    else:
        org_price.append("N/A")
        
        
        
    
    if sale_price_elements[i] is not None:
        sale_price.append(sale_price_elements[i].text.strip())
    else:
        sale_price.append("N/A")
        
        
        
    
    if new_badge_elements[i] is not None:
        new_badge.append(new_badge_elements[i].text.strip())
    else:
        new_badge.append("Old")
        
        
        
        
    if availability_elements[i] is not None:
        availability.append(availability_elements[i].text.strip())
    else:
        availability.append("N/A")
        

        
    if availability[i] == 'Quick Add' :
        if prod_name_elements[i] is not None :
            color.append(prod_name_elements[i].text.split()[2])
        else:
            color.append("N/A")
    else:
        color.append("Out Of Stock")

                
#print(len(color))
#print(color)
        
    
    
#-------------------------------------Getting The Underlined Links From prod_name_elements----------------------------

    if prod_name_elements[i] is not None:
        link.append("https://townteam.com/"+prod_name_elements[i].attrs["href"])
    else:
        link.append("N/A")


#print(len(link))
#print(link)






#------------------------------Getting The Internal Information from  The Link Container Created------------------------
    if availability[i] == 'Quick Add' :
        if link[i] != "N/A":
            shoe_page = requests.get(link[i],headers=headers)
            #Page Source
            shoe_src = shoe_page.content
            #Parsing
            shoe_soup = BeautifulSoup(shoe_src, "lxml")
            size_elements = shoe_soup.find("fieldset",{"class":"js product-form__input clearfix","data-product-attribute":"set-rectangle"})
            size.append(size_elements.text.split()[1])

        else:
            size.append("N/A")
    else:
        size.append("Out Of Stock")

#print(size)  














In [152]:
print("products_elements: ",len(products))
print("prod_names_elements: ",len(prod_name_elements))
print("org_prices_elements: ",len(org_price_elements))
print("sale_prices_elements: ",len(sale_price_elements))
print("new_badges_elements: ",len(new_badge_elements))
print("availability_elements: ",len(availability_elements))

products_elements:  15
prod_names_elements:  15
org_prices_elements:  15
sale_prices_elements:  15
new_badges_elements:  15
availability_elements:  15


In [156]:
# -------------------------------------------------SAVE THE DATA---------------------------------------------------
## Create CSV File & Fill It

file_list = [prod_name,org_price,sale_price,new_badge,availability,size,color,link]
extended_file_list = zip_longest(*file_list)

with open("E:/The Journey/Career\Data Science/_Portfolio Projects/Python Projects/Web Scraping Projects/Town Team/TownTeamScraping.csv", "w",encoding='utf-8', newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(["Product_ID","Original Price","Sale Price","Badge","Availability","Size","Color","Link"])
    wr.writerows(extended_file_list)